## Notebook for CodeSpace

#### Init Git

In [ ]:
## Basic import of modules
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
## data acquisition
## using kaggle dataset
movies_df=pd.read_csv('movies_df.csv')

### Columns to be dropped
* Budget
* Homepage
* Original_language
* Original_title
* Popularity
* Production_companies
* Production_countries
* Release_date (Good if you want to make it in time period)
* Revenue
* Runtime
* Spoken_Language
* Released ??
* Tagline
* Vote_avg
* movie_id

In [ ]:
movies_df.info()

In [ ]:
credit_df=pd.read_csv('tmdb_5000_credits.csv')

## Merge two dataframes

In [ ]:
movies_df=movies_df.merge(credit_df,on='title')

In [ ]:
# Extracting useful columns
movies_df=movies_df[['id','title','genres','overview','keywords','cast','crew']]

In [ ]:
# Check for duplicates and drop empty cols
movies_df.isna().sum()
movies_df.duplicated().sum()

### The genre keywords are not in specific format we need to make it a proper data structure

In [ ]:
# sample code
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
# Function that helps to extract the keys of the genre col like ['action','thriller',...] and same for keywords col
def genre_converter(str_list):
    gnr_list=[]
    li=ast.literal_eval(str_list)
    for i in li:
        gnr_list.append(i['name'])
    return gnr_list 

In [ ]:
movies_df['genres']=movies_df['genres'].apply(genre_converter)
movies_df['keywords']=movies_df['keywords'].apply(genre_converter)

In [ ]:
# Selecting the top 3 actors of the movie
def cast_selection(cast_list):
    gnr_list=[]
    li=ast.literal_eval(cast_list)
    if (len(li)>=3):
        for i in range(3):
            gnr_list.append(li[i]['name'])
    else:
        if(len(li)>0 and len(li)<3):
            for i in range(len(li)):
                gnr_list.append(li[i]['name'])
    return gnr_list

In [ ]:
movies_df['cast']=movies_df['cast'].apply(cast_selection)

In [ ]:
# Extracting the director name
def director_exct(crew_list):
    li=ast.literal_eval(crew_list)
    di=[]
    for i in li:
        if(i['job']=='Director'):
            di.append(i['name'])
           
       # print('Job : ',i['job'])
    return di

In [ ]:
movies_df['crew']=movies_df['crew'].apply(director_exct)

In [ ]:
# Now tokenize the overview column
from nltk.tokenize import word_tokenize

In [ ]:
movies_df.overview=movies_df.overview.apply(lambda x:x .split())

# Now we need to do imp part
### In every element of list two words separated by a space need to be joined so that it is treated as a same word

In [ ]:
movies_df.crew=movies_df.crew.apply(lambda x :[i.replace(" ",'') for i in x])
movies_df.genres=movies_df.genres.apply(lambda x :[i.replace(" ",'') for i in x])
movies_df.cast=movies_df.cast.apply(lambda x :[i.replace(" ",'') for i in x])
movies_df.keywords=movies_df.keywords.apply(lambda x :[i.replace(" ",'') for i in x])

## Add the imp columns into single df

In [ ]:
movies_df['tags']=movies_df['overview']+movies_df['genres']+movies_df['keywords']+movies_df['cast']+movies_df['crew']

## tags col modification

In [ ]:
movies_df.drop(columns=['genres','overview','keywords','cast','crew'],inplace=True)
movies_df['tags']=movies_df['tags'].apply(lambda x : " ".join(x))
movies_df['tags']=movies_df['tags'].apply(lambda x : x.lower())

## Creating model for Bag of words technique for vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(stop_words='english',max_features=5000)

In [ ]:
vectors=cv.fit_transform(movies_df.tags).toarray()
vectors.shape

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stemmer_(text):
    txt=[]
    for i in text.split():
        txt.append(ps.stem(i))
    return " ".join(txt)  

In [ ]:
movies_df['tags']=movies_df['tags'].apply(stemmer_)

In [ ]:
cv.get_feature_names_out()

## To know that wether a given vector is how much similar to a given array and how closely related 
### we will be using sklearn's cosine similarity 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)
similarity.shape

In [ ]:
def find_best_movie(movie):
    index_len=len(movies_df[movies_df['title']==movie])
    if(index_len!=0):
        index_pos=movies_df[movies_df['title']==movie].index[0]
        # the similarity[] is an 2d array for finding similarity
        best_set=sorted(enumerate(similarity[index_pos]),reverse=True,key=lambda x:x[1])[1:6]
        for i in best_set:
            print(movies_df.iloc[i[0]]['title'])
    else:
        print(['No movies found'])
    #print(best_set)